In [29]:
import _MEWtools as mt
import multiprocessing

## Get baseline time for CPU Intensive Workload

In [17]:
iterations = 5

def slow_worker():
    for i in range(8000):
        square = i**i

    print('Done')

In [18]:
import time

t0 = time.time()

for i in range(iterations):
    slow_worker()

t1 = time.time()

total = t1-t0
display(f'Total Execution Time: {total} seconds')

Done
Done
Done
Done
Done


'Total Execution Time: 11.490794897079468 seconds'

## Perform same operation with multiprocessing

In [19]:
t0 = time.time()

pool = multiprocessing.Pool(iterations)
for i in range(iterations):
    pool.apply_async(slow_worker)
    
pool.close()
pool.join()

t1 = time.time()

total = t1-t0
display(f'Total Execution Time: {total} seconds')

Done
Done
Done
Done
Done


'Total Execution Time: 2.5882318019866943 seconds'

## Load an interior structure from a config file and populate a MEWTools Satellite

In [20]:
import utils

overrides = {
    'obliquity': 0,
    'obliquityPhase': 0,
    'obliquityPhaseRate': 0,
    'spinRate': 0,
    'nonSynchronusRotationRate': 0,
    'librationAmplitude': 0,
    'librationPhase': 0,
    'librationFrequency': 0
}

sample_sat = utils.import_structure('Sample', overrides)

In [21]:
structures = utils.list_structures()
structures

Sample


['Sample']

In [22]:
import sympy as sym
import numpy as np

r, θ, φ, t = sym.symbols('r θ φ t', real = True)
sample_sat.tt.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()

-22175.3871428071 + 9.90608959403468*I

In [8]:
ttR = sym.re(sample_sat.tt.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand())
ppR = sym.re(sample_sat.pp.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand())
tpR = sym.re(sample_sat.tp.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand())

import time

diffs = []
for i in range(100000):
    t0 = time.time()

    val1 = sample_sat.PC1.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()
    val2 = sample_sat.PC2.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()
    val3 = sample_sat.PCΨ.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()
    val4 = sample_sat.PCΨ2.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()

    t1 = time.time()

    total1 = t1-t0
#     display(f'Total Execution Time: {total1} seconds')

    import math
    t0 = time.time()

    pc1  = (1/2) * (ttR + ppR + math.sqrt(4*tpR**2 + (ttR-ppR)**2))
    pc2  = (1/2) * (ttR + ppR - math.sqrt(4*tpR**2 + (ttR-ppR)**2))
    pc3  = (1/2) * math.atan( (2*tpR)/(ttR-ppR))
    pc4 = (1/2) * math.atan2((2*tpR),(ttR-ppR))

    t1 = time.time()

    total2 = t1-t0
#     display(f'Total Optimized Execution Time: {total2} seconds')
    diffs.append(total1 - total2)
    
np.average(diffs)

0.00013730335712432861

In [9]:
print(val1)
print(pc1)

30660.6119811987
30660.6119811986


In [10]:
ttR

-22175.3871428071

## Checking degrees to radians performance with constant vs equation

In [23]:
import time
import numpy as np

ITERATIONS = 10000000

t0 = time.time()

for i in range(ITERATIONS):
    rads = 60 * (np.pi / 180)

t1 = time.time()

total = t1-t0
display(f'Rads: {rads}')
display(f'Total Full Execution Time: {total} seconds')

# With constant
t0 = time.time()

rad_multiplier = np.pi / 180
for i in range(ITERATIONS):
    rads = 60 * rad_multiplier

t1 = time.time()

total = t1-t0
display(f'Rads: {rads}')
display(f'Total Execution Time: {total} seconds')

'Rads: 1.0471975511965976'

'Total Full Execution Time: 1.2331218719482422 seconds'

'Rads: 1.0471975511965976'

'Total Execution Time: 0.7603740692138672 seconds'

**Theta = Longitude**

**Phi = Latitude**

## Lets try to build a grid

In [42]:
import pandas as pd

sat = utils.import_structure('Sample', overrides)

TIME_STEPS = 1
MIN_LAT = -90
MAX_LAT = 90
MIN_LON = -180
MAX_LON = -170
RAD_MULTIPLIER = np.pi / 180

data = []


def callback(stress_item):
    data.append(stress_item)
    

def get_stress(step, lat, lon):
    lat_radians = lat * RAD_MULTIPLIER
    lon_radians = lon * RAD_MULTIPLIER

    principal1 = sat.PC1.subs(t,step).subs(φ,lat_radians).subs(θ,lon_radians).expand()
    principal2 = sat.PC2.subs(t,0).subs(φ,lat_radians).subs(θ,lon_radians).expand()
    principal_phi = sat.PCΨ.subs(t,0).subs(φ,lat_radians).subs(θ,lon_radians).expand()
    principal_phi2 = sat.PCΨ2.subs(t,0).subs(φ,lat_radians).subs(θ,lon_radians).expand()

    return {
                'time_step': step,
                'latitude': lat,
                'latitude_radians': lat_radians,
                'longitude': lon,
                'longitude_radians': lon_radians,
                'principal1': principal1,
                'principal2': principal2,
                'principal_phi': principal_phi,
                'principal_phi2': principal_phi2
            }

    
pool = multiprocessing.Pool(12)

t0 = time.time()

for step in range(TIME_STEPS):
    for lat in range(MIN_LAT, MAX_LAT + 1):
#         lat_radians = lat * RAD_MULTIPLIER

        for lon in range(MIN_LON, MAX_LON + 1):
#             lon_radians = lon * RAD_MULTIPLIER
            pool.apply_async(get_stress, args = (step, lat, lon, ), callback=callback)
            
pool.close()
pool.join()

t1 = time.time()
total = (t1 - t0)
           
print(f'Calculation time: {total}')
df = pd.DataFrame(data)        
display(df)

Calculation time: 7.2163050174713135


,time_step,latitude,latitude_radians,longitude,longitude_radians,principal1,principal2,principal_phi,principal_phi2
0,0,-90,-1.570796,-180,-3.141593,16970.4496767830,-88701.5485712285,6.12323399573677e-17,6.12323399573677e-17 - 0.5*pi
1,0,-90,-1.570796,-177,-3.089233,16970.4496767830,-88701.5485712285,6.11484231635600e-17,6.114842316356e-17 - 0.5*pi
2,0,-90,-1.570796,-175,-3.054326,16970.4496767830,-88701.5485712285,6.09993324172810e-17,6.0999332417281e-17 - 0.5*pi
3,0,-90,-1.570796,-178,-3.106686,16970.4496767830,-88701.5485712285,6.11950388703081e-17,6.11950388703081e-17 - 0.5*pi
4,0,-90,-1.570796,-179,-3.124139,16970.4496767830,-88701.5485712285,6.12230139754066e-17,6.12230139754066e-17 - 0.5*pi
...,...,...,...,...,...,...,...,...,...
1986,0,90,1.570796,-174,-3.036873,16970.4496767830,-88701.5485712285,-6.08969027922358e-17,-6.08969027922358e-17 + 0.5*pi
1987,0,90,1.570796,-175,-3.054326,16970.4496767830,-88701.5485712285,-6.09993324172810e-17,-6.0999332417281e-17 + 0.5*pi
1988,0,90,1.570796,-173,-3.019420,16970.4496767830,-88701.5485712285,-6.07759233806784e-17,-6.07759233806784e-17 + 0.5*pi
1989,0,90,1.570796,-172,-3.001966,16970.4496767830,-88701.5485712285,-6.06364310341097e-17,-6.06364310341097e-17 + 0.5*pi


In [46]:
import pandas as pd

sat = utils.import_structure('Sample', overrides)

TIME_STEPS = 1
MIN_LAT = -90
MAX_LAT = 90
MIN_LON = -180
MAX_LON = 180
RAD_MULTIPLIER = np.pi / 180

data = []


def callback(stress_items):
    data.extend(stress_items)
    

def get_stress_for_lat(step, lat):
    results = []
    lat_radians = lat * RAD_MULTIPLIER
    
    for lon in range(MIN_LON, MAX_LON + 1, 15):
        lon_radians = lon * RAD_MULTIPLIER

        principal1 = sat.PC1.subs(t,step).subs(φ,lat_radians).subs(θ,lon_radians).expand()
        principal2 = sat.PC2.subs(t,0).subs(φ,lat_radians).subs(θ,lon_radians).expand()
        principal_phi = sat.PCΨ.subs(t,0).subs(φ,lat_radians).subs(θ,lon_radians).expand()
        principal_phi2 = sat.PCΨ2.subs(t,0).subs(φ,lat_radians).subs(θ,lon_radians).expand()

        results.append({
                'time_step': step,
                'latitude': lat,
                'longitude': lon,
                'principal1': principal1,
                'principal2': principal2,
                'principal_phi': principal_phi,
                'principal_phi2': principal_phi2
            })
        
    return results

    
pool = multiprocessing.Pool(12)

t0 = time.time()

for step in range(TIME_STEPS):
    for lat in range(MIN_LAT, MAX_LAT + 1, 15):
        pool.apply_async(get_stress_for_lat, args = (step, lat, ), callback=callback)
            
pool.close()
pool.join()
            
t1 = time.time()
total = (t1 - t0)
           
print(f'Calculation time: {total}')

df = pd.DataFrame(data)        
df

Calculation time: 0.6377549171447754


,time_step,latitude,longitude,principal1,principal2,principal_phi,principal_phi2
0,0,0,-180,16970.4496767830,-88701.5485712285,0,0
1,0,0,-165,20638.7176105022,-77954.5989892249,0,0
2,0,0,-150,30660.6119811986,-48593.3867048100,0,0
3,0,0,-135,44350.7742856143,-8485.22483839148,0,0
4,0,0,-120,58040.9365900299,31622.9370280271,0,0
...,...,...,...,...,...,...,...
320,0,90,120,16970.4496767830,-88701.5485712285,-3.06161699786838e-17,-3.06161699786838e-17 + 0.5*pi
321,0,90,135,16970.4496767830,-88701.5485712285,-4.32978028117747e-17,-4.32978028117747e-17 + 0.5*pi
322,0,90,150,16970.4496767830,-88701.5485712285,-5.30287619362453e-17,-5.30287619362454e-17 + 0.5*pi
323,0,90,165,16970.4496767830,-88701.5485712285,-5.91458985689335e-17,-5.91458985689335e-17 + 0.5*pi
